In [1]:
import sys
sys.path.append('..') 
from src.vdb_managers.chroma_manager import ChromaManager

In [2]:
chroma_store = ChromaManager() # 自定义的chroma类

In [4]:
file_path = "files/论文 - GraphRAG.pdf"
chunk_size = 1000
chunk_overlap = 100
auto_extract_metadata = True
metadata = {
    'keywords': "实习", # 必须是str，好像bool等也行，不记得了，反正不能是list
    "from": "上齐实习"
}
collection_name = "RAG"
discription = "test vdb function"
summarize_chunks = True
similarity_metric = "cosine"

In [5]:
# 上传PDF文档
chroma_store.upload_pdf_file(file_path = file_path, collection_name = collection_name, summarize_chunks=summarize_chunks)

In [5]:
collection_name = "RAG"
query = "What are the general methods of deep reinforcement learning?"
k = 3
metadata_filter = {'关键字key': {'$in': ['graphrag', 'node', 'edge']}}
fuzzy_filter = True
use_summary = True # 是否使用summary进行匹配，还是使用原始文本进行匹配

In [ ]:
results = chroma_store.dense_search(collection_name=collection_name, query=query, k=k)

In [ ]:
type(results), results.keys(), results['metadatas']

In [ ]:
dense_weight = 0.5

In [ ]:
results = chroma_store.hybrid_search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

In [ ]:
results = chroma_store.search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

In [ ]:
formatted_context = chroma_store.get_formatted_context(collection_name=collection_name, query=query, k=k, dense_weight=dense_weight)
formatted_context

### 2. help

In [1]:
import os
from dotenv import load_dotenv # type: ignore

load_dotenv()
api_key = os.getenv('ZETATECHS_API_KEY')
base_url = os.getenv('ZETATECHS_API_BASE')

import chromadb # type: ignore
import chromadb.utils.embedding_functions as embedding_functions # type: ignore

client = chromadb.PersistentClient(path="ChromaVDB")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=api_key,
    api_base=base_url,
    model_name="text-embedding-3-large"
)

collection = client.get_or_create_collection(name="test", embedding_function=openai_ef)

In [ ]:
help(collection.query)

In [ ]:
help(client.get_or_create_collection)

In [ ]:
help(collection.add)

In [ ]:
help(client.create_collection)